In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import LabelEncoder
import re

### Reading exported data from Telegram

In [ ]:
data = pd.read_json('result.json')
data = pd.DataFrame(data['chats']['list'])

In [ ]:
def union(data):
    for i in range(1,len(data)):
        if data['from'][i] == data['from'][i-1] and data['time_after_prev'][i] <= 15:
            data.text[i] = data.text[i-1]+'. '+data.text[i]
            data.time_after_prev[i] = data.time_after_prev[i]+data.time_after_prev[i-1]
            data['duplicate'][i-1] = 1 
    return data


def resp(data):
    for i in range(1,len(data)):
        if data.time_after_prev[i]<900 and data['from'][i]!=data['from'][i-1]:
            data.response[i-1]=data.text[i]
    return data


def preprocess(data, cols):
    data.drop(cols, axis=1, inplace=True)
    data.drop(data.loc[data.type=='service'].index, inplace=True)
    data.drop(data.loc[data.text==''].index, inplace=True)
    data.date_unixtime=data.date_unixtime.astype('int64')

    data=data.reset_index(drop=True)
    data['time_after_prev'] = 0

    for i in range(1,len(data)):
        data['time_after_prev'][i] = data['date_unixtime'][i]-data['date_unixtime'][i-1]

    data.drop(data.loc[data.text.apply(type) != str].index, inplace=True)
    data=data.reset_index(drop=True)
    data['duplicate'] = 0

    data = union(data)

    data.drop(data.loc[data.duplicate==1].index, inplace=True)
    data.drop('duplicate',axis=1, inplace=True)
    data=data.reset_index(drop=True)
    data['response'] = 0

    data = resp(data)
    data.drop(data.loc[data['from']=='Mary Lazarenko'].index, inplace=True)
    data.drop(data.loc[data['response']==0].index, inplace=True)
    data.drop(['date_unixtime','type','time_after_prev'], axis=1,inplace=True)
    data=data.reset_index(drop=True)

    return data

---
I will work with the most numerous dialogues from my collection. These are the ones I had with my family and closest friends.

Unfortunately, json file I exported from TG is pretty inconsistent and it has different sets of fields for each dialogue, so at this point I will edit them individually.

I will drop all messages containing files, gifs, links, images etc. so that only texts remain. Then they all messages will be organized in "message" and "response" columns based on sender info time difference between "message" and "response".  

---

In [ ]:
maximka=pd.DataFrame((data['messages'][1]))
aisha = pd.DataFrame((data['messages'][4]))
lilya = pd.DataFrame((data['messages'][9]))
katya = pd.DataFrame((data['messages'][10]))
lyalya = pd.DataFrame((data['messages'][11]))
dad = pd.DataFrame((data['messages'][17]))
mom = pd.DataFrame((data['messages'][19]))

In [ ]:
max_cols = ['date','from_id','sticker_emoji','width','height','mime_type','duration_seconds','reply_to_message_id','actor','actor_id','action','discard_reason','file','thumbnail','media_type','photo','forwarded_from','edited','edited_unixtime','via_bot','game_title','game_description','game_link','game_message_id','score']
maximka=preprocess(maximka, max_cols)


sos_cols = ['date','from_id','forwarded_from', 'file', 'thumbnail', 'media_type', 'sticker_emoji',
       'width', 'height', 'edited', 'edited_unixtime', 'photo', 'mime_type',
       'duration_seconds', 'reply_to_message_id','actor','actor_id','action','discard_reason','location_information','emoticon','live_location_period_seconds']
aisha=preprocess(aisha, sos_cols)


lil_cols = ['date','from_id','forwarded_from', 'file', 'thumbnail', 'media_type', 'sticker_emoji',
       'width', 'height', 'edited', 'edited_unixtime', 'photo', 'mime_type','duration_seconds', 'reply_to_message_id']
lilya = preprocess(lilya, lil_cols)


kat_cols = ['date','from_id', 'photo',
       'width', 'height', 'file', 'thumbnail', 'media_type',
       'mime_type', 'duration_seconds', 'sticker_emoji', 'forwarded_from',
       'reply_to_message_id', 'edited', 'edited_unixtime', 'actor', 'actor_id',
       'action', 'discard_reason', 'contact_information', 'contact_vcard']
katya = preprocess(katya, kat_cols)


lya_col = ['date','from_id',
       'forwarded_from', 'file', 'thumbnail', 'media_type', 'mime_type',
       'duration_seconds', 'width', 'height', 'photo', 'actor',
       'actor_id', 'action', 'discard_reason', 'sticker_emoji',
       'reply_to_message_id', 'edited', 'edited_unixtime']
lyalya = preprocess(lyalya,lya_col)


dad_cols = ['date', 'actor', 'actor_id', 'action','from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'edited', 'edited_unixtime',
       'forwarded_from', 'photo', 'contact_information', 'contact_vcard',
       'reply_to_message_id']
dad = preprocess(dad,dad_cols)


mom_cols = ['date','actor', 'actor_id', 'action', 'from_id', 'file', 'media_type', 'mime_type',
       'duration_seconds', 'photo', 'width', 'height', 'thumbnail',
       'sticker_emoji', 'forwarded_from', 'edited', 'edited_unixtime',
       'reply_to_message_id', 'discard_reason']
mom = preprocess(mom,mom_cols)

In [ ]:
# Union all datasets in one

frames = [maximka,aisha,lilya,katya,lyalya,dad,mom]
full_data = pd.concat(frames)
full_data=full_data.reset_index(drop=True)

---
Some messages have a lot of punctuation, emojis and other symbols, so my next move will be to remove them all from the data

---

In [ ]:
def process(data):

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
  
    for i in range(len(data)):
        data['text'][i] = data['text'][i].lower().replace('\n', ' ') \
        .replace('-', ' ').replace(':', ' ').replace(')', '').replace('))', '').replace('))))', '') \
        .replace(',', '').replace('"', '') \
        .replace("...", ".").replace("..", ".").replace("....", ".") \
        .replace("!", ".").replace("?", "") \
        .replace(";", ".").replace(":", " ")

    data['text'][i]=emoji_pattern.sub(r'', data['text'][i])

    data['text'][i] = "".join(v for v in data['text'][i] if v not in string.punctuation).lower()
    data['text'][i] = " ".join(data['text'][i].split())

    data['response'][i] = data['response'][i].lower().replace('\n', ' ') \
    .replace('-', ' ').replace(':', ' ').replace(')', '').replace('))', '').replace('))))', '') \
    .replace(',', '').replace('"', '') \
    .replace("...", ".").replace("..", ".").replace("....", ".") \
    .replace("!", ".").replace("?", "") \
    .replace(";", ".").replace(":", " ")

    data['response'][i]=emoji_pattern.sub(r'', data['response'][i])

    data['response'][i] = "".join(v for v in data['response'][i] if v not in string.punctuation).lower()
    data['response'][i] = " ".join(data['response'][i].split())

    return data

full_data.drop('id',axis=1,inplace=True)

le = LabelEncoder()
full_data['from'] = le.fit_transform(full_data['from'])

full_data = process(full_data)

In [ ]:
# Saving clear data 
full_data.to_csv('punc_preprocessed.csv', index=False)